<a href="https://colab.research.google.com/github/JunhanBai/Pratice/blob/main/Prompt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Prompt templates (what you'd send to an LLM) ---
CLASSIFY_PROMPT = """
You are a support assistant. Classify the customer's intent into one of:
[order_status, refund_request, product_question, other]
Customer message: "{message}"
Return only the intent.
"""

EXTRACT_PROMPT = """
You are a data extractor for support tickets.
Given intent "{intent}" and message "{message}",
return a JSON-like dict with keys: order_id (if any), product (if any), sentiment (positive/neutral/negative).
"""

RESPOND_PROMPT = """
You are a friendly support agent. Using details {details} and intent "{intent}",
write a short, empathetic reply (<= 80 words) with a clear next step.
"""

# --- Minimal "mock LLM" so this runs without API keys ---
import re

def mock_llm_classify(message: str) -> str:
    m = message.lower()
    if any(k in m for k in ["refund","return","exchange","money back"]): return "refund_request"
    if any(k in m for k in ["track","where is","hasn't arrived","order","package"]): return "order_status"
    if any(k in m for k in ["size","color","compatible","spec","does it work with"]): return "product_question"
    return "other"

def mock_llm_extract(intent: str, message: str) -> dict:
    # naive order id: five digits or "#12345"
    m = re.search(r"(?:order|#)\s*(\d{5})", message, flags=re.I)
    order_id = m.group(1) if m else None
    product = None
    pm = re.search(r"\b(my|the)\s+([A-Za-z0-9\- ]{3,30})", message)
    if pm: product = pm.group(2).strip()
    sentiment = "negative" if any(w in message.lower() for w in ["angry","upset","annoy","late","broken","hasn't"]) else "neutral"
    return {"order_id": order_id, "product": product, "sentiment": sentiment}

def mock_llm_respond(intent: str, details: dict) -> str:
    if intent == "order_status":
        if details.get("order_id"):
            return (f"Thanks for checking in! I found order #{details['order_id']}. "
                    "It’s in transit; deliveries can take up to 5 business days. "
                    "If it still hasn’t arrived after that, reply here and I’ll escalate.")
        return "Happy to help with tracking. Could you share your 5-digit order number so I can look it up?"
    if intent == "refund_request":
        prod = details.get("product") or "item"
        return (f"I’m sorry your {prod} isn’t working out. We accept returns within 30 days. "
                "Please share your order number and I’ll send a prepaid return label.")
    if intent == "product_question":
        return ("Great question! Could you share the product model and how you plan to use it? "
                "I’ll confirm compatibility and recommend the best option.")
    return "Thanks for reaching out! Tell me a bit more about what you need and I’ll point you in the right direction."

# --- Orchestrator (the chain) ---
def run_chain(message: str) -> dict:
    intent = mock_llm_classify(message)  # Step 1
    details = mock_llm_extract(intent, message)  # Step 2
    reply = mock_llm_respond(intent, details)  # Step 3
    # Keep the filled prompts so you can see the chain "inputs"
    return {
        "filled_prompts": {
            "classify": CLASSIFY_PROMPT.format(message=message),
            "extract": EXTRACT_PROMPT.format(intent=intent, message=message),
            "respond": RESPOND_PROMPT.format(intent=intent, details=details),
        },
        "intent": intent,
        "details": details,
        "reply": reply
    }

# --- Quick demos ---
examples = [
    "Hi, my order #10234 hasn't arrived yet. Can you check?",
    "I want to return my headphones—they stopped working.",
    "Is the charger compatible with the X200 laptop?"
]

for msg in examples:
    out = run_chain(msg)
    print("\nMessage:", msg)
    print("Intent:", out["intent"])
    print("Details:", out["details"])
    print("Reply:", out["reply"])



Message: Hi, my order #10234 hasn't arrived yet. Can you check?
Intent: order_status
Details: {'order_id': '10234', 'product': 'order', 'sentiment': 'negative'}
Reply: Thanks for checking in! I found order #10234. It’s in transit; deliveries can take up to 5 business days. If it still hasn’t arrived after that, reply here and I’ll escalate.

Message: I want to return my headphones—they stopped working.
Intent: refund_request
Details: {'order_id': None, 'product': 'headphones', 'sentiment': 'neutral'}
Reply: I’m sorry your headphones isn’t working out. We accept returns within 30 days. Please share your order number and I’ll send a prepaid return label.

Message: Is the charger compatible with the X200 laptop?
Intent: product_question
Details: {'order_id': None, 'product': 'charger compatible with the X2', 'sentiment': 'neutral'}
Reply: Great question! Could you share the product model and how you plan to use it? I’ll confirm compatibility and recommend the best option.
